# Lab | BabyAGI with agent - autonomous agents 

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

- BabyAGI is like a mini AI agent that:

Has a goal/objective (e.g. "Find the best research papers on Mars rovers")

Breaks that goal into smaller tasks

Runs each task using tools (like Google search, LLMs, databases)

Updates its memory and creates the next task

Keeps looping until the job is done

## Install and Import Required Modules

In [8]:
!pip install langchain_community langchain_experimental langchain_openai

In [9]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
# THE MAIN CHARACTER — BabyAGI is the core agent running
from langchain_openai import OpenAI, OpenAIEmbeddings


## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [10]:
 #%pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import faiss
# memory stored as vectors. FAISS is a fast search engine 

In [11]:
!pip install faiss-cpu


# 🔐 Loading Environment Variables

In [12]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY') 

#  Initialized FAISS Vectorstore (Memory System)

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [14]:
vectorstore = FAISS(embedding_function=embeddings_model, index=index, docstore=InMemoryDocstore({}), index_to_docstore_id={})
# The vectorstore is a FAISS index that stores the embeddings of the documents.

In [15]:
from langchain_community.utilities import SerpAPIWrapper
search = SerpAPIWrapper(serpapi_api_key="3f175694f016a43c563a0b9439e11403def7872575b9a20794c6543678088f01")

In [16]:
!pip install google-search-results

 # BabyAGI Chains: Planning How It Thinks
# Three core chains:
 1. Task creator 
 2. Task prioritizer 
 3. Execution chain (agent) 


In [17]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate


todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [18]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

C:\Users\44758\AppData\Local\Temp\ipykernel_16156\3064747660.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


### 👩‍🚀 Space-Themed Objective:

In [19]:
OBJECTIVE = "Act as a space analyst and generate a mission briefing on upcoming rocket launches and weather conditions relevant to aerospace operations today."


In [20]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [21]:
baby_agi({"objective": OBJECTIVE})

C:\Users\44758\AppData\Local\Temp\ipykernel_16156\3867971396.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  baby_agi({"objective": OBJECTIVE})



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I need to understand the objective for the todo list
Action: TODO
Action Input: objective
Observation: 

1. Define the objective: The first step in creating a todo list is to clearly define the objective. This will help guide all the tasks and actions that need to be completed.

2. Set a timeline: Determine a realistic timeline for completing the objective. This will help prioritize tasks and ensure that everything is completed in a timely manner.

3. Identify key tasks: Break down the objective into smaller, manageable tasks. This will make it easier to tackle and ensure that nothing is overlooked.

4. Assign responsibilities: Determine who will be responsible for each task. This will help ensure accountability and make sure that all tasks are completed.

5. Gather necessary resources: Identify any resources that will be needed to complete the objective.

{'objective': 'Act as a space analyst and generate a mission briefing on upcoming rocket launches and weather conditions relevant to aerospace operations today.'}